In [12]:
import openai
import json

client = openai.OpenAI()
messages = []

In [11]:
def get_weather(city):
    return "33 deegress celcius."

FUNCTION_MAP = {
    "get_weather": get_weather
}

In [ ]:
# 모델에게 함수들의 설명을 전달하기 위함.

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of",
                    }
                },
                "required": ["city"],
            },
        }
    }
]

In [ ]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls > 0:
        # 기록을 남기는 정도의 역할을 하는 로직
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                } for tool_call in message.tool_calls
            ]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            function_args = tool_call.function.arguments

            print(f"Calling function : {function_name} with {function_args}")
            result = "Not implemented"
            try:
                args = json.loads(function_args)
                function_to_run = FUNCTION_MAP.get(function_name)
                result = function_to_run(**args)
            except json.JSONDecodeError as e:
                print(f"Error decoding function arguments: {e}")
                result += str(e)
                args = {}
            except Exception as e:
                print(f"Error calling function {function_name}: {e}")
                result += str(e)

            # 함수 호출 결과를 메시지에 추가 (결과 기록)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })

In [ ]:
def call_ai():
    response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=TOOLS
        )
    # 모델은 도구를 사용하고 싶을때, message.content 안에 write 하지 않는다.
    process_ai_response(response.choices[0].message)
    print(response)
    message = response.choices[0].message.tool_calls
    messages.append({"role": "assistant", "content": message})
    print(f"Assistant: {message}")

while True:
    message = input("Send a message to the LLM")
    if message == "quit" or message == 'q':
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()
        

User: My name is luke
Assistant: Nice to meet you, Luke! How can I assist you today?
User: What is my name ?
Assistant: Your name is Luke.
User: Im from korea
Assistant: That's great, Luke! Korea has a rich culture and history. Are you from South Korea or North Korea?
User: What was the first question i asked you and what is the closest Island country to where I was born ?
Assistant: The first question you asked me was, "What is my name?" 

As for the closest island country to Korea, it's Japan. Japan is located to the east of the Korean Peninsula, across the Korea Strait. If you were born in Korea, Japan is the nearest island nation.
User: 
Assistant: It looks like your message might have been cut off. How can I assist you further?


KeyboardInterrupt: Interrupted by user